In [1]:
from sklearn.metrics.pairwise import pairwise_distances
from tensorflow.keras.utils import multi_gpu_model
from model import identity_block, conv_block
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import layers
import numpy as np
import h5py
from os.path import isfile, join
import os
import matplotlib.pyplot as plt

In [2]:
lr=1e-4
epochs=250
decay=1e-2

data_path="/home/kjakkala/neuralwave/data/CSI_48_mc"
data_files = [join(data_path, f) for f in os.listdir(data_path) if isfile(join(data_path, f)) and f.endswith(".h5")]
history = []

hf = h5py.File(data_files[0], 'r')
train_classes = np.array(hf.get('labels')).astype(str)
num_classes = train_classes.shape[0]
X_train = np.expand_dims(hf.get('X_train'), axis=-1)
X_test = np.expand_dims(hf.get('X_test'), axis=-1)
y_train = np.eye(num_classes)[hf.get('y_train')]
y_test =np.eye(num_classes)[hf.get('y_test')]
hf.close()
    
input_layer = layers.Input(shape=(X_train.shape[1:]))

x = layers.Conv2D(64, (7, 7), strides=(2, 2))(input_layer)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, [64, 256], "relu")
x = identity_block(x, [64, 256], "relu")

x = conv_block(x, [128, 512], "relu")
x = identity_block(x, [128, 512], "relu")

x = conv_block(x, [256, 1024], "relu")
x = identity_block(x, [256, 1024], "relu")

x = conv_block(x, [512, 2048], "relu")
x = identity_block(x, [512, 2048], "relu")

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(num_classes, activation=None)(x)
x = layers.Activation('softmax')(x)

model_base = Model(inputs=input_layer, outputs=x)
model_base.summary()
model = multi_gpu_model(model_base, gpus=4)

for data_ind in range(len(data_files)):
    print("MC:{}".format(data_ind))

    hf = h5py.File(data_files[data_ind], 'r')
    train_classes = np.array(hf.get('labels')).astype(str)
    num_classes = train_classes.shape[0]
    X_train = np.expand_dims(hf.get('X_train'), axis=-1)
    X_test = np.expand_dims(hf.get('X_test'), axis=-1)
    y_train = np.eye(num_classes)[hf.get('y_train')]
    y_test =np.eye(num_classes)[hf.get('y_test')]
    hf.close()

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr, decay=decay), metrics=['acc'])
    history.append(model.fit(x=X_train, y=y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=2))

MC:0
(1631, 2000, 270, 1) (288, 2000, 270, 1) (1631, 49) (288, 49)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2000, 270, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 997, 132, 64) 3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 997, 132, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 997, 132, 64) 0           batch_normalization[0][0]        
__________________________________________

Train on 1631 samples, validate on 288 samples
Epoch 1/250
 - 45s - loss: 3.4647 - acc: 0.1858 - val_loss: 5.0784 - val_acc: 0.0208
Epoch 2/250
 - 32s - loss: 1.6426 - acc: 0.5819 - val_loss: 7.1695 - val_acc: 0.0243
Epoch 3/250
 - 32s - loss: 0.8315 - acc: 0.7854 - val_loss: 1.0923 - val_acc: 0.7257
Epoch 4/250
 - 32s - loss: 0.4977 - acc: 0.8712 - val_loss: 0.5927 - val_acc: 0.8576
Epoch 5/250
 - 32s - loss: 0.3517 - acc: 0.8927 - val_loss: 0.4856 - val_acc: 0.8819
Epoch 6/250
 - 32s - loss: 0.2724 - acc: 0.9178 - val_loss: 0.4007 - val_acc: 0.9062
Epoch 7/250
 - 32s - loss: 0.2236 - acc: 0.9424 - val_loss: 0.3916 - val_acc: 0.8889
Epoch 8/250
 - 32s - loss: 0.1830 - acc: 0.9479 - val_loss: 0.3973 - val_acc: 0.8993
Epoch 9/250
 - 32s - loss: 0.1615 - acc: 0.9571 - val_loss: 0.3677 - val_acc: 0.9062
Epoch 10/250
 - 32s - loss: 0.1287 - acc: 0.9632 - val_loss: 0.3488 - val_acc: 0.9167
Epoch 11/250
 - 32s - loss: 0.1380 - acc: 0.9644 - val_loss: 0.3542 - val_acc: 0.9097
Epoch 12/250
 - 

Epoch 96/250
 - 32s - loss: 0.0068 - acc: 1.0000 - val_loss: 0.3637 - val_acc: 0.9236
Epoch 97/250
 - 32s - loss: 0.0087 - acc: 0.9988 - val_loss: 0.3495 - val_acc: 0.9201
Epoch 98/250
 - 32s - loss: 0.0074 - acc: 1.0000 - val_loss: 0.3475 - val_acc: 0.9201
Epoch 99/250
 - 32s - loss: 0.0143 - acc: 0.9982 - val_loss: 0.3679 - val_acc: 0.9201
Epoch 100/250
 - 32s - loss: 0.0080 - acc: 0.9982 - val_loss: 0.3429 - val_acc: 0.9236
Epoch 101/250
 - 33s - loss: 0.0097 - acc: 0.9975 - val_loss: 0.3472 - val_acc: 0.9236
Epoch 102/250
 - 32s - loss: 0.0076 - acc: 0.9994 - val_loss: 0.3500 - val_acc: 0.9236
Epoch 103/250
 - 32s - loss: 0.0094 - acc: 0.9988 - val_loss: 0.3446 - val_acc: 0.9271
Epoch 104/250
 - 32s - loss: 0.0087 - acc: 1.0000 - val_loss: 0.3543 - val_acc: 0.9236
Epoch 105/250
 - 32s - loss: 0.0089 - acc: 0.9988 - val_loss: 0.3615 - val_acc: 0.9236
Epoch 106/250
 - 32s - loss: 0.0079 - acc: 0.9988 - val_loss: 0.3611 - val_acc: 0.9236
Epoch 107/250
 - 32s - loss: 0.0066 - acc: 1.00

Epoch 191/250
 - 32s - loss: 0.0055 - acc: 0.9994 - val_loss: 0.3809 - val_acc: 0.9271
Epoch 192/250
 - 32s - loss: 0.0036 - acc: 1.0000 - val_loss: 0.3793 - val_acc: 0.9271
Epoch 193/250
 - 32s - loss: 0.0044 - acc: 0.9994 - val_loss: 0.3758 - val_acc: 0.9306
Epoch 194/250
 - 32s - loss: 0.0039 - acc: 0.9994 - val_loss: 0.3929 - val_acc: 0.9306
Epoch 195/250
 - 32s - loss: 0.0041 - acc: 0.9994 - val_loss: 0.3843 - val_acc: 0.9306
Epoch 196/250
 - 32s - loss: 0.0033 - acc: 1.0000 - val_loss: 0.3817 - val_acc: 0.9271
Epoch 197/250
 - 32s - loss: 0.0033 - acc: 0.9994 - val_loss: 0.3793 - val_acc: 0.9306
Epoch 198/250
 - 32s - loss: 0.0032 - acc: 1.0000 - val_loss: 0.3837 - val_acc: 0.9306
Epoch 199/250
 - 32s - loss: 0.0063 - acc: 0.9994 - val_loss: 0.3745 - val_acc: 0.9306
Epoch 200/250
 - 32s - loss: 0.0028 - acc: 1.0000 - val_loss: 0.3750 - val_acc: 0.9306
Epoch 201/250
 - 32s - loss: 0.0027 - acc: 1.0000 - val_loss: 0.3788 - val_acc: 0.9306
Epoch 202/250
 - 32s - loss: 0.0029 - acc: 

Train on 1631 samples, validate on 288 samples
Epoch 1/250
 - 44s - loss: 3.5001 - acc: 0.1882 - val_loss: 5.2051 - val_acc: 0.0208
Epoch 2/250
 - 36s - loss: 1.6688 - acc: 0.5935 - val_loss: 6.4421 - val_acc: 0.0243
Epoch 3/250
 - 36s - loss: 0.9065 - acc: 0.7578 - val_loss: 1.2501 - val_acc: 0.6806
Epoch 4/250
 - 36s - loss: 0.5520 - acc: 0.8473 - val_loss: 0.4629 - val_acc: 0.8854
Epoch 5/250
 - 36s - loss: 0.3976 - acc: 0.8921 - val_loss: 0.3672 - val_acc: 0.8924
Epoch 6/250
 - 36s - loss: 0.2908 - acc: 0.9172 - val_loss: 0.2953 - val_acc: 0.9062
Epoch 7/250
 - 36s - loss: 0.2370 - acc: 0.9295 - val_loss: 0.2781 - val_acc: 0.9167
Epoch 8/250
 - 36s - loss: 0.1917 - acc: 0.9448 - val_loss: 0.2372 - val_acc: 0.9271
Epoch 9/250
 - 36s - loss: 0.1714 - acc: 0.9522 - val_loss: 0.2298 - val_acc: 0.9167
Epoch 10/250
 - 36s - loss: 0.1527 - acc: 0.9601 - val_loss: 0.2157 - val_acc: 0.9514
Epoch 11/250
 - 36s - loss: 0.1307 - acc: 0.9651 - val_loss: 0.2062 - val_acc: 0.9444
Epoch 12/250
 - 

...

Train on 1631 samples, validate on 288 samples
Epoch 1/250
 - 44s - loss: 3.5225 - acc: 0.1766 - val_loss: 4.9559 - val_acc: 0.0208
Epoch 2/250
 - 35s - loss: 1.8217 - acc: 0.5518 - val_loss: 5.5160 - val_acc: 0.0243
Epoch 3/250
 - 35s - loss: 0.9169 - acc: 0.7603 - val_loss: 1.0795 - val_acc: 0.7049
Epoch 4/250
 - 35s - loss: 0.5689 - acc: 0.8535 - val_loss: 0.4591 - val_acc: 0.8542
Epoch 5/250
 - 35s - loss: 0.3844 - acc: 0.8982 - val_loss: 0.3522 - val_acc: 0.8958
Epoch 6/250
 - 35s - loss: 0.3127 - acc: 0.9117 - val_loss: 0.3170 - val_acc: 0.9271
Epoch 7/250
 - 35s - loss: 0.2548 - acc: 0.9350 - val_loss: 0.2938 - val_acc: 0.9236
Epoch 8/250
 - 35s - loss: 0.2048 - acc: 0.9424 - val_loss: 0.2915 - val_acc: 0.9132
Epoch 9/250
 - 35s - loss: 0.1740 - acc: 0.9522 - val_loss: 0.2781 - val_acc: 0.9306
Epoch 10/250
 - 35s - loss: 0.1452 - acc: 0.9608 - val_loss: 0.2385 - val_acc: 0.9444
Epoch 11/250
 - 35s - loss: 0.1338 - acc: 0.9601 - val_loss: 0.2400 - val_acc: 0.9444
Epoch 12/250
 - 

Epoch 96/250
 - 35s - loss: 0.0123 - acc: 0.9988 - val_loss: 0.2329 - val_acc: 0.9653
Epoch 97/250
 - 36s - loss: 0.0092 - acc: 0.9988 - val_loss: 0.2311 - val_acc: 0.9653
Epoch 98/250
 - 35s - loss: 0.0108 - acc: 0.9994 - val_loss: 0.2294 - val_acc: 0.9653
Epoch 99/250
 - 35s - loss: 0.0123 - acc: 0.9988 - val_loss: 0.2372 - val_acc: 0.9653
Epoch 100/250
 - 35s - loss: 0.0140 - acc: 0.9969 - val_loss: 0.2411 - val_acc: 0.9618
Epoch 101/250
 - 35s - loss: 0.0089 - acc: 1.0000 - val_loss: 0.2335 - val_acc: 0.9688
Epoch 102/250
 - 35s - loss: 0.0087 - acc: 0.9994 - val_loss: 0.2395 - val_acc: 0.9653
Epoch 103/250
 - 35s - loss: 0.0068 - acc: 1.0000 - val_loss: 0.2372 - val_acc: 0.9653
Epoch 104/250
 - 35s - loss: 0.0107 - acc: 0.9982 - val_loss: 0.2383 - val_acc: 0.9688
Epoch 105/250
 - 35s - loss: 0.0091 - acc: 0.9988 - val_loss: 0.2378 - val_acc: 0.9688
Epoch 106/250
 - 35s - loss: 0.0109 - acc: 0.9975 - val_loss: 0.2384 - val_acc: 0.9653
Epoch 107/250
 - 35s - loss: 0.0075 - acc: 0.99

Epoch 191/250
 - 35s - loss: 0.0049 - acc: 1.0000 - val_loss: 0.2527 - val_acc: 0.9688
Epoch 192/250
 - 35s - loss: 0.0070 - acc: 0.9994 - val_loss: 0.2583 - val_acc: 0.9583
Epoch 193/250
 - 35s - loss: 0.0045 - acc: 0.9994 - val_loss: 0.2545 - val_acc: 0.9653
Epoch 194/250
 - 35s - loss: 0.0035 - acc: 0.9994 - val_loss: 0.2563 - val_acc: 0.9688
Epoch 195/250
 - 35s - loss: 0.0066 - acc: 0.9994 - val_loss: 0.2501 - val_acc: 0.9722
Epoch 196/250
 - 35s - loss: 0.0085 - acc: 0.9988 - val_loss: 0.2632 - val_acc: 0.9653
Epoch 197/250
 - 35s - loss: 0.0059 - acc: 0.9994 - val_loss: 0.2561 - val_acc: 0.9653
Epoch 198/250
 - 35s - loss: 0.0033 - acc: 1.0000 - val_loss: 0.2576 - val_acc: 0.9688
Epoch 199/250
 - 35s - loss: 0.0049 - acc: 0.9994 - val_loss: 0.2529 - val_acc: 0.9653
Epoch 200/250
 - 35s - loss: 0.0045 - acc: 0.9994 - val_loss: 0.2540 - val_acc: 0.9653
Epoch 201/250
 - 35s - loss: 0.0036 - acc: 1.0000 - val_loss: 0.2549 - val_acc: 0.9653
Epoch 202/250
 - 35s - loss: 0.0045 - acc: 

Train on 1631 samples, validate on 288 samples
Epoch 1/250
 - 46s - loss: 3.6959 - acc: 0.1901 - val_loss: 4.8900 - val_acc: 0.0208
Epoch 2/250
 - 36s - loss: 1.9480 - acc: 0.5500 - val_loss: 7.6690 - val_acc: 0.0208
Epoch 3/250
 - 36s - loss: 0.9912 - acc: 0.7388 - val_loss: 1.1920 - val_acc: 0.7049
Epoch 4/250
 - 36s - loss: 0.6384 - acc: 0.8375 - val_loss: 0.5706 - val_acc: 0.8333
Epoch 5/250
 - 36s - loss: 0.4323 - acc: 0.8921 - val_loss: 0.4224 - val_acc: 0.8889
Epoch 6/250
 - 36s - loss: 0.3302 - acc: 0.9154 - val_loss: 0.3586 - val_acc: 0.8924
Epoch 7/250
 - 36s - loss: 0.2613 - acc: 0.9289 - val_loss: 0.3128 - val_acc: 0.9097
Epoch 8/250
 - 36s - loss: 0.2295 - acc: 0.9375 - val_loss: 0.3171 - val_acc: 0.9097
Epoch 9/250
 - 36s - loss: 0.1942 - acc: 0.9418 - val_loss: 0.3064 - val_acc: 0.9236
Epoch 10/250
 - 36s - loss: 0.1771 - acc: 0.9448 - val_loss: 0.2642 - val_acc: 0.9201
Epoch 11/250
 - 36s - loss: 0.1646 - acc: 0.9516 - val_loss: 0.2815 - val_acc: 0.9236
Epoch 12/250
 - 

ResourceExhaustedError: OOM when allocating tensor with shape[32,64,997,132] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_63/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 2, 2], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_63/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_63/Conv2D/ReadVariableOp)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
data_dir="/home/kjakkala/neuralwave/data/CSI_intruder_pca.h5"

hf = h5py.File(data_dir, 'r')
classes = np.array(hf.get('labels')).astype(str)
X_data = np.expand_dims(hf.get('X_data'), axis=-1)
y_data = np.eye(classes.shape[0])[hf.get('y_data')]
hf.close()

print(X_data.shape)

In [ ]:
for i in range(X_data.shape[0]):
    print("{:16}{}".format(train_classes[np.argmax(model.predict(X_data), axis=-1)[i]], classes[np.argmax(y_data, axis=-1)[i]]))